In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install Sentencepiece
#!pip install transformers

In [ ]:
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from transformers import (AdamW, 
                          T5ForConditionalGeneration,
                          T5Tokenizer,
                          get_linear_schedule_with_warmup)

In [ ]:
cd '/content/drive/MyDrive/T5 QnA/QNA'

/content/drive/MyDrive/T5 QnA/QNA


In [ ]:
from dataset_pipeline import BioQADataset
from data_extraction import extract_data

In [ ]:
path1 = 'data/BioASQ/BioASQ-train-factoid-6b.json'
path2 = 'data/BioASQ/BioASQ-train-factoid-5b.json'
path3 = 'data/BioASQ/BioASQ-train-factoid-6b.json'

In [ ]:
df_lst = []
for path in [path1, path2, path3]:
  df_lst.append(extract_data(path))

data = pd.concat(df_lst)

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.05, random_state=20)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

train_set = BioQADataset(train_df, tokenizer)
train_loader = DataLoader(train_set, batch_size=8)

test_set = BioQADataset(test_df, tokenizer)
test_loader = DataLoader(test_set, batch_size=8)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small', from_tf=False, return_dict=True)
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=0.0001)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
#forward pass with full data
from tqdm import tqdm
for epoch in range(5):
  batch_loss = 0
  losses, preds, labels = [], [], []
  for batch in tqdm(train_loader):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    
    optimizer.zero_grad()

    output = model(input_ids=input_ids,
                  attention_mask=attention_mask,
                  labels=labels)
    batch_loss = output.loss
    losses.append(batch_loss.item())    

    batch_loss.backward()
    optimizer.step()



  print('Train loss:', np.sum(losses)/len(train_loader))

100%|██████████| 1722/1722 [06:18<00:00,  4.55it/s]


Train loss: 0.21499262538802907


100%|██████████| 1722/1722 [06:18<00:00,  4.56it/s]


Train loss: 0.06991421707928003


100%|██████████| 1722/1722 [06:19<00:00,  4.53it/s]


Train loss: 0.053936131825588395


100%|██████████| 1722/1722 [06:18<00:00,  4.54it/s]


Train loss: 0.050721364006906876


100%|██████████| 1722/1722 [06:19<00:00,  4.54it/s]

Train loss: 0.0429735710142593


In [ ]:
torch.save(model.state_dict(), 'model.pt')

In [ ]:
model2 = T5ForConditionalGeneration.from_pretrained('t5-small', from_tf=False, return_dict=True)
model2.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>